In [5]:
import numpy as np
import nglview as nv
import MDAnalysis as mda

u = mda.Universe("../data/aldp/aldp.pdb")

name = 'mle'
positions = np.load(f"../assets/aldp/{name}/samples.npy")[2].reshape(-1, 3)

new_u = mda.Merge(u.atoms)
new_u.atoms.positions = positions * 10

view = nv.show_mdanalysis(new_u)
view.add_representation("ball+stick")
view

NGLWidget()

In [6]:
view.download_image(filename=f'../figure/aldp_mle_2.png')

In [112]:
from PIL import Image

# Load the saved image
image_path = '../figure/aldp_ours.png'
image = Image.open(image_path)

# Crop the image to a square
width, height = image.size
min_dimension = min(width, height)
left = (width - min_dimension) // 2
top = (height - min_dimension) // 2
right = left + min_dimension
bottom = top + min_dimension

cropped_image = image.crop((left, top, right, bottom))

# Save the cropped image
cropped_image.save('../figure/aldp_ours_cropped.png')

In [11]:
from PIL import Image, ImageDraw
import os
import matplotlib.pyplot as plt

plt.rcParams.update({
    'font.family': 'serif',
})

# 기본 제목 매핑 및 순서
default_titles = {
    'ref': 'Reference',
    'ours': 'SGDS',
    'mle': 'MLE',
    'pis': 'PIS',
    'tb': 'TB',
    'tb_expl_ls': 'TB+Expl+LS'
}
order = ['ref', 'ours', 'mle', 'pis', 'tb', 'tb_expl_ls']

def create_mosaic(
    base_dir: str,
    output_path: str,
    line_width: int = 4,
    line_color=(0,0,0),
    font_path: str = None,
    font_size: int = 16
):
    """
    base_dir 아래 각 하위 디렉토리에서 첫 3개의 PNG를
    중앙 크롭 후 3×n 모자이크를 만들고,
    헤더 없이 전체 경계선만 그립니다.
    """
    # 1. 하위 디렉토리 목록 (사용자 지정 순서 유지)
    order = ['ref', 'ours', 'mle', 'pis', 'tb', 'tb_expl_ls']
    dirs = [d for d in order if os.path.isdir(os.path.join(base_dir, d))]
    if not dirs:
        raise RuntimeError(f"하위 디렉토리가 없습니다: {base_dir}")
    
    # 2. 각 폴더에서 첫 3개 PNG 경로 수집
    per_dir_paths = {}
    all_imgs = []
    for d in dirs:
        folder = os.path.join(base_dir, d)
        pngs = sorted(f for f in os.listdir(folder) if f.lower().endswith('.png'))
        if len(pngs) < 3:
            raise RuntimeError(f"'{d}' 폴더에 PNG 파일이 3개 미만입니다.")
        paths = [os.path.join(folder, p) for p in pngs[:3]]
        per_dir_paths[d] = paths
        all_imgs.extend(paths)

    # 3. 크롭 사이즈 결정 (가장 작은 변 길이)
    min_side = min(min(Image.open(p).size) for p in all_imgs)
    tile = min_side
    n_cols = len(dirs)
    
    # 4. 캔버스 크기 (헤더 없이 이미지 영역만)
    width = tile * n_cols
    height = tile * 3
    mosaic = Image.new('RGB', (width, height), (255,255,255))
    draw = ImageDraw.Draw(mosaic)

    # 5. 각 이미지 중앙 크롭해서 붙이기
    for col, d in enumerate(dirs):
        for row, img_path in enumerate(per_dir_paths[d]):
            img = Image.open(img_path)
            w, h = img.size
            left = (w - tile) // 2
            top  = (h - tile) // 2
            crop = img.crop((left, top, left + tile, top + tile))
            mosaic.paste(crop, (col * tile, row * tile))
    
    # 6. 구분선 그리기
    # 세로선 (컬럼 사이)
    for col in range(1, n_cols):
        x = col * tile
        draw.line([(x, 0), (x, height)], fill=line_color, width=line_width)
    # 가로선 (행 사이)
    for row in range(1, 3):
        y = row * tile
        draw.line([(0, y), (width, y)], fill=line_color, width=line_width)

    # 전체 외곽선
    draw.rectangle([0, 0, width, height], outline=line_color, width=line_width)

    # 7. 저장
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    mosaic.save(output_path)
    print(f"✔ 저장 완료: {output_path}")
    
def save_header_image(
    output_path: str,
    font_size: int = 20,
    tile_width: int = 200
):
    """
    기본 순서대로 컬럼 제목만 저장합니다. PPT 캡처용.
    """
    titles_list = [default_titles[key] for key in order]
    n = len(titles_list)
    fig, ax = plt.subplots(
        figsize=(n * tile_width / 100, tile_width / 100)
    )
    ax.axis('off')
    for i, title in enumerate(titles_list):
        if title == 'SGDS':  # 강조할 제목
            ax.text(
                (i + 0.5) / n,
                0.5,
                title,
                ha='center',
                va='center',
                fontsize=font_size,
                # fontweight='bold',  # 굵게
                style='italic',     # 이탤릭체
                # color='red'         # 강조 색상 (선택 사항)
            )
        else:
            ax.text(
                (i + 0.5) / n,
                0.5,
                title,
                ha='center',
                va='center',
                fontsize=font_size
            )
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    print(f"✔ 헤더 이미지 저장 완료: {output_path}")

# 사용 예
base_dir    = "../assets/aldp/3d"
output_path = "../figure/3d.png"
create_mosaic(base_dir, output_path)
save_header_image('../figure/header_only.png')


✔ 저장 완료: ../figure/3d.png
✔ 헤더 이미지 저장 완료: ../figure/header_only.png
